Обучение проводилось в Yandex Data Sphere в конфигурации g2.1 с ускорителем A100

In [ ]:
# %pip install transformers==4.39.1
# %pip install peft==0.10.0
# %pip install sentencepiece==0.2.0
# %pip install accelerate==0.28.0
# %pip install -i https://pypi.org/simple/ bitsandbytes

# %pip install --force-reinstall transformers==4.38.2

# Нужно достаточно места на виртуалке
# - 15Гб это система и кеш базовой модели, 
# - 13.5Гб на каждую сохраненную модель
# - 0.5Гб на чекпоинты на каждую модель 
# Всего 40Гб хватало тык в притык на 2 модели

Обучение сделано на основе этого примера <br>
[Подробный разбор дообучения (fine-tuning) LLaMa (на примере задачи генерации заголовков к новостям)](https://youtu.be/zHv5pA-lxAA)<br>



In [1]:
import pickle
import numpy as np
import torch
import transformers
from transformers import LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from peft import LoraConfig, get_peft_model, PeftModel
import pandas as pd

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
OUTPUT_DIR = "model_checkpoints"
FINETUNED_DIR = "model_finetuned"

# Инициализация основной модели

In [2]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BASE_MODEL = "NousResearch/Llama-2-7b-hf"

# Инициализируем токенизатор для модели LLaMa
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)

# задаем токен для padding, то есть добавления в те последовательности из батча, которые короче,
# чем максимальная длина последовательности, чтобы все последовательности в итоге были одной длины
tokenizer.pad_token_id = 0

# нули для padding будем добавлять слева
tokenizer.padding_side = "left"

In [3]:
# Инициализируем основную модель

"""
model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    #load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto",
)
"""

'\nmodel = LlamaForCausalLM.from_pretrained(\n    BASE_MODEL,\n    #load_in_8bit=True,\n    torch_dtype=torch.float16,\n    device_map="auto",\n)\n'

In [4]:
# Инициализируем основную модель
# Если мы хотим обучать модель в int4 для уменьшения требуемой видеопамяти

model = LlamaForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    #load_in_4bit=True,
    #load_in_8bit=False,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4', # квантизация модели в тип normal float 4
    )
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.02s/it]
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.


In [5]:
model.config.pad_token_id = tokenizer.pad_token_id = 0  # unk
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [6]:
model = model.eval()

#!!! комментирую только при квантизации
# model = torch.compile(model)

# Загрузка и предобработка данных

In [7]:
Xy_train =  pd.read_csv('Xy_train.csv', index_col='Unnamed: 0')
Xy_test =  pd.read_csv('Xy_test.csv', index_col='Unnamed: 0')

In [14]:
# у нас датасет с уже подготовленными промптами

def generate_prompt(sample):
    # Также для разделения частей промпта можно использовать специальные токены начала и конца сегмента <s> и </s>
    prompt = sample['prompt']
    full_prompt = sample['prompt'] + ' ' + sample['reference']

    return prompt, full_prompt

In [15]:
def tokenize(prompt, full_prompt, add_eos_token=True):
    result = tokenizer(
        full_prompt,
        padding=False,
        return_tensors=None,
    )
    if (
            result["input_ids"][-1] != tokenizer.eos_token_id
            #and len(result["input_ids"]) < CUTOFF_LEN
            and add_eos_token
    ):
        # если в конце пос-ти нет специального токена, мы его добавляем
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    prompt_len = len(tokenizer(prompt, padding=False, return_tensors=None)["input_ids"])
    labels = result["input_ids"].copy()
    labels = [-100 for _ in range(prompt_len)] + labels[prompt_len:]
    result["labels"] = labels

    return result

In [16]:
def generate_and_tokenize_prompt(sample):
    prompt, full_prompt = generate_prompt(sample)
    tokenized_full_prompt = tokenize(prompt, full_prompt)
    return tokenized_full_prompt

In [17]:
train_data = list(Xy_train.apply(generate_and_tokenize_prompt, axis=1))
test_data = list(Xy_test.apply(generate_and_tokenize_prompt, axis=1))


In [22]:
# data_collator нужен для формирования батча (padding, сборка элементов батча в один тензор,
# конвертация массивов numpy или списков в тензоры torch.LongTensor)
data_collator = transformers.DataCollatorForSeq2Seq(
    tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True, label_pad_token_id=-100
)

2024-03-27 07:34:34.623164: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# Добавляем в LLaMa адаптеры

In [23]:
# Размерность матриц адаптеров
# К примеру, если исходная матрица весов 4096 x 4096, то матрицы, которые мы добавляем,
# имеют размерность 4096 х LORA_R и LORA_R х 4096.
LORA_R = 8

# После умножения на матрицу весов адаптеров компоненты вектора делим на LORA_R и умножаем на LORA_ALPHA
LORA_ALPHA = 16
LORA_DROPOUT = 0.05

# В какие слои трансформера будем добавлять адаптеры, в данном случае - в матрицы в слоях self-attention
# для вычисления query и key.
LORA_TARGET_MODULES = [
    "q_proj",
    "v_proj",
]

# Делаем объект конфигурации по параметрам адаптеров
config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=LORA_TARGET_MODULES,
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

In [24]:
model = get_peft_model(model, config)

# Выведем информацию об обучаемых весах модели.
# model.print_trainable_parameters()

# Задаем гиперпараметры обучения

In [25]:
BATCH_SIZE = 4
TRAIN_EPOCHS = 3
MICRO_BATCH_SIZE = 1 #2
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4

training_arguments = transformers.TrainingArguments(
    per_device_train_batch_size=MICRO_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    warmup_steps=100,
    max_steps=500,
    num_train_epochs=TRAIN_EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=True,
    logging_steps=50,
    optim="adamw_torch",
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=50,
    save_steps=50,
    output_dir=OUTPUT_DIR,
    save_total_limit=10,
    load_best_model_at_end=True,
    report_to="none"
)

# Запускаем обучение

In [26]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=training_arguments,
    data_collator=data_collator
)

/home/jupyter/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [27]:
model.config.use_cache = False

# компиляция модели (для оптимизации обучения)
model = torch.compile(model)

In [28]:
# torch.set_float32_matmul_precision('high')

In [29]:
# Обучение
trainer.train()

  5%|▌         | 100/2000 [01:51<35:25,  1.12s/it]

{'loss': 0.288, 'grad_norm': 1.089714527130127, 'learning_rate': 0.00029699999999999996, 'epoch': 0.71}



 97%|█████████▋| 34/35 [00:24<00:00,  1.33it/s]
                                                  
100%|██████████| 35/35 [00:25<00:00,  1.39it/s]
                                               

{'eval_loss': 0.18123137950897217, 'eval_runtime': 26.2412, 'eval_samples_per_second': 10.594, 'eval_steps_per_second': 1.334, 'epoch': 0.71}


 10%|█         | 200/2000 [04:10<33:45,  1.13s/it]  

...


 97%|█████████▋| 34/35 [00:24<00:00,  1.33it/s]
                                                   
100%|██████████| 35/35 [00:25<00:00,  1.39it/s]
                                               

{'eval_loss': 0.31293630599975586, 'eval_runtime': 26.2507, 'eval_samples_per_second': 10.59, 'eval_steps_per_second': 1.333, 'epoch': 14.23}


100%|██████████| 2000/2000 [46:18<00:00,  1.39s/it]

{'train_runtime': 2778.8397, 'train_samples_per_second': 2.879, 'train_steps_per_second': 0.72, 'train_loss': 0.03658888978511095, 'epoch': 14.23}


TrainOutput(global_step=2000, training_loss=0.03658888978511095, metrics={'train_runtime': 2778.8397, 'train_samples_per_second': 2.879, 'train_steps_per_second': 0.72, 'train_loss': 0.03658888978511095, 'epoch': 14.23})

## Сохраняем чекпоинты обученной модели

In [30]:
model.save_pretrained(OUTPUT_DIR)

## Сохранение объединенной модели (LLM+LORA)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL,
        return_dict=True,
        torch_dtype=torch.float16
    )

base_model.do_sample = False
base_model.temperature = None

In [ ]:
model = PeftModel.from_pretrained(base_model, OUTPUT_DIR, torch_dtype=torch.float16,
    #load_in_4bit=True,
    #load_in_8bit=False,
    device_map="auto",
    temperature = 0,
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4', # квантизация модели в тип normal float 4
    ))

model.do_sample = False
model.temperature = None

In [ ]:
model = model.merge_and_unload()
model.do_sample = False
model.temperature = None
model.generation_config = GenerationConfig(
        temperature=None,
        do_sample = False
    )

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [ ]:
model.save_pretrained(FINETUNED_DIR)

In [ ]:
tokenizer.save_pretrained(FINETUNED_DIR)

## Конвертация в gguf 

git clone https://github.com/ggerganov/llama.cpp.git

python -m venv venv

source venv/bin/activate

pip install -r llama.cpp/requirements.txt

python llama.cpp/convert.py -h

python llama.cpp/convert.py model_finetuned --outfile Llama-2-7b-ft-f16.gguf --outtype f16
 
## Квантизация

make -C llama.cpp -j4

llama.cpp/quantize Llama-2-7b-ft-f16.gguf Llama-2-7b-ft-q4.gguf Q4_0